In [5]:
#IMPORT MODULES

import cdflib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import math
import datetime
from scipy.fft import fftshift, fftfreq, fft
from scipy.integrate import trapezoid
import calendar
import time
from plasmapy.formulary import lower_hybrid_frequency
from astropy import units as u

#FILE SETUP
#Input: CDF file path, brst variable to extract

files = []
files.append(["../20170810data/mms1/fgm/mms1_fgm_brst_l2_20170810121733_v5.99.0.cdf", "mms1_fgm_b_gsm_brst_l2"])
files.append(["../20170810data/mms2/fgm/mms2_fgm_brst_l2_20170810121733_v5.99.0.cdf", "mms2_fgm_b_gsm_brst_l2"])
files.append(["../20170810data/mms3/fgm/mms3_fgm_brst_l2_20170810121733_v5.99.0.cdf", "mms3_fgm_b_gsm_brst_l2"])
files.append(["../20170810data/mms4/fgm/mms4_fgm_brst_l2_20170810121733_v5.99.0.cdf", "mms4_fgm_b_gsm_brst_l2"])

In [6]:
#FUNCTION DEFINITIONS

def readvariables(vartype):
    '''
    Not used, testing function.
    Extracts and sorts CDF data into lists and returns it
      Inputs:
          vartype- r or z, for rVariables or zVariables
      Outputs:
          returnlist- list of variables in CDF file
    '''
    returnlist = [] #returns this list
    for i in file.cdf_info().get(vartype + "Variables"):
        try:
            print(i + " " + str(file.varinq(i).get("Last_Rec"))) #Print name of variable and number of records
            returnlist.append([])
            for j in range(file.varinq(i).get("Last_Rec")):
                returnlist[file.cdf_info().get(vartype + "Variables").index(i)].append(file.varget(i,startrec=j,endrec=j+1).tolist())
                #Append data to the right list and right index
        except ValueError:
            print("No records found") #In case there are no records
            continue
    return returnlist

def filenames_get(name_list_file):
  '''
  Not used, testing function.
  Pulls list of filenames I'm using from the file where they are stored.
  Allows some flexibility
  Inputs:
      name_list_file- string which constains the full path to the file which
          contains a list of the full filename paths needed
  Outputs:
      name_list- list of strings which contain the full path to
          each file
  '''
  name_list=[]
  with open(name_list_file,"r") as name_file_obj: #read-only access
       for line in name_file_obj:
           line_clean =line.rstrip('\n') #removes newline chars from lines
           name_list.append(line_clean)
  return name_list

def get_cdf_var(filename,varnames):
  '''
  Privately used function.
  pulls particular variables from a CDF
  note: if variable has more than one set of data (E.g. b-field with x,y,z
  components) it will be necessary to format the data by reshaping the array
  from a 1D to a 2D array
  (may find workaround/better way later)
  Inputs:
      filename- string of the complete path to the specified CDF file
      varnames- list of strings which contain the CDF variables that are
          to be extracted
  Outputs:
      data- list of numpy arrays containing the desired variables' data
  '''
  cdf_file=cdflib.CDF(filename,varnames)
  data=[]
  for varname in varnames:
      var_data=np.array(cdf_file.varget(varname))
      data.append(var_data)
  return data

def getData(file, timeInterval, axis):
    '''
    Gets relevant processed data from CDF file.
    Inputs:
        file- filename taken from array "files"
        timeInterval- period for data
        axis- String value, "x","y","z", or "all" (a vector summation of all components)
    Outputs:
        x- horizontal axis data, in this case Epoch (ns)
        y- veritcal axis data, in this case B-field measurement (nT)
    '''
    CDFfile = cdflib.CDF(file[0])
    filename = file[0]
    varname = file[1]

    #generate time series
    raw_times = get_cdf_var(filename, ["Epoch"])[0]
    times = []
    
    #time of event
    year = 2017
    month = 8
    day = 10
    hour = 12

    start_minute = timeInterval[0]
    start_sec = timeInterval[1]
    stop_minute = timeInterval[2]
    stop_sec = timeInterval[3]
    
    #algorithm to find indexes of given period start and end, quickly
    #Note that 'epoch' is 1/1/2000 12:00:00 (MMS epoch) J2000, also known as TT2000
    start_epoch_sec = calendar.timegm((year-30, month, day+1, hour-12, start_minute+1, start_sec+9))
    stop_epoch_sec = calendar.timegm((year-30, month, day+1, hour-12, stop_minute+1, stop_sec+9))
    start_index_epoch_sec = raw_times[0] / 1e9
    stop_index_epoch_sec = raw_times[-1] / 1e9
    time_between_measurements = (stop_index_epoch_sec - start_index_epoch_sec) / (len(raw_times) - 1)
    est_start_index = round((start_epoch_sec - start_index_epoch_sec) / time_between_measurements)
    est_stop_index = round((stop_epoch_sec - start_index_epoch_sec) / time_between_measurements)
    est_start_indexes = np.arange(est_start_index - 5, est_start_index + 5, 1).tolist()
    est_stop_indexes = np.arange(est_stop_index - 5, est_stop_index + 5, 1).tolist()
    
    for i in est_start_indexes:
        i_epoch_sec = round(raw_times[i] / 1e9)
        if i_epoch_sec == start_epoch_sec:
            start_index = i
    for j in est_stop_indexes:
        j_epoch_sec = round(raw_times[j] / 1e9)
        if j_epoch_sec == stop_epoch_sec:
            stop_index = j

    raw_data = get_cdf_var(filename, [varname])[0]

    if axis == "all": #vector addition
        Bx = []
        By = []
        Bz = []
        
        for i in range(0,len(raw_data)):
            Bx.append(raw_data[i][0]) # x data
            By.append(raw_data[i][1]) # y data
            Bz.append(raw_data[i][2]) # z data

        Bmag = vectorAdd(Bx, By, Bz)
        
        #data sets as arrays, len = 2048
        x = raw_times[start_index:stop_index]
        y = Bmag[start_index:stop_index]
        
    else:
        data = []
            
        coordinateSystem = {"x":0, "y":1, "z":2}
        coordinate = coordinateSystem.get(axis)

        for i in range(0,len(raw_data)):
            data.append(raw_data[i][coordinate])

        #data sets as arrays, len = 2048
        x = raw_times[start_index:stop_index]
        y = data[start_index:stop_index]
        print(y)

    return x, y

def getFFTdata(x, y):
    '''
    Takes raw x and y data, generates FFT plot.
    Inputs:
        x- horizontal axis data, in this case Epoch (ns)
        y- veritcal axis data, in this case B-field measurement (nT)
    Outputs:
        xf- horizontal axis FFT data, in this case frequency (Hz)
        yf- vertical axis FFT data, in this case power (nT^2)
    '''
    N = len(x)
    T = (x[1] - x[0])/1e9 #~7.8 millisecond timestep

    #square amplitude to obtain power
    y2 = []
    for i in range(0, len(y)):
        y2.append(pow(y[i], 2))
        
    #average value
    sum = 0
    for i in range(0, N):
        sum += y2[i]

    average = sum/N
    
    #generate FFT
    yf = fftshift(fft(y2))
    xf = fftshift(fftfreq(N, T))

    return xf, yf, N

def createTicks(time_interval, epoch, num_of_ticks=None):
    '''
    Matplotlib plots nanosecond TT2000 time series data. This function creates ticks in UTC to designate the x-axis.
    Inputs:
        time_interval- period
        epoch- nanosecond TT2000 time series
    Outputs:
        tick_indexes- array[0], index of tick positions on x-axis
        tick_values- array[1], String value of tick labels on x-axis
    '''
    if num_of_ticks == None: #If not specified count 1 second per tick
        num_of_ticks = time_interval[3] - time_interval[1] + 1
    tick_indexes = []
    tick_values = np.linspace(time_interval[1], time_interval[3], num_of_ticks).tolist()

    distance_between_ticks = math.floor((len(epoch)/(num_of_ticks-1)))
    for x in range(num_of_ticks):
        if distance_between_ticks*x >= 1:
            tick_indexes.append(epoch[distance_between_ticks*x-1])
        else:
            tick_indexes.append(epoch[distance_between_ticks*x])
    return [tick_indexes, tick_values]

def powerFFT(period, xf, yfSeries, limit, lowerORupper):
    '''
    Computes the power from the signal by integrating the FFT plot against frequency.
    Inputs:
        period- relevant period
        xf- FFT formatted x-axis data, as produced by getFFTdata()
        yfSeries- array of all FFT formatted y-axis data from all 4 MMS spacecraft
        limit- the bound of integration, in this case the lower hybrid frequency
        lowerORupper- either "upperLimit" or "lowerLimit", whether the limit should be an upper or lower bound
    Outputs:
        res- value of the integral of power against frequency in the given bounds
    '''
    N = len(yfSeries[0])
    
    T = period[3]-period[1] #ONLY WORKS for time series values in the same minute
    deltaF = 1/T
    
    avgYf = []
    
    if lowerORupper == "upperLimit": # values f < fLH
        for i in range(10, limit): #ignore low frequencies as background reconnection, not wave fluctuation 
            Yval = (abs(yfSeries[0][i]) + abs(yfSeries[1][i]) + abs(yfSeries[2][i]) + abs(yfSeries[3][i]))/4
            avgYf.append(Yval/deltaF)
        xf = xf[10:limit]
    else: #"lowerLimit", values f > fLH
        for i in range(limit, N): 
            Yval = (abs(yfSeries[0][i]) + abs(yfSeries[1][i]) + abs(yfSeries[2][i]) + abs(yfSeries[3][i]))/4
            avgYf.append(Yval/deltaF)
        xf = xf[limit:]

    res = trapezoid(avgYf, xf) #trapezoidal integration
    
    return res

def vectorAdd(Vx, Vy, Vz):
    '''
    Computes vector addition of the given components in the XYZ directions.
    Inputs:
         Vx- x-component
         Vy- y-component
         Vz- z-component
    Outputs:
        Vmag- the magnitude of the resultant vector of the XYZ components
    '''
    N = len(Vx)
    Vmag = []
    
    for i in range(0, N):
        mag = math.sqrt(pow(Vx[i],2) + pow(Vy[i],2) + pow(Vz[i],2))
        Vmag.append(mag)
        
    return Vmag

def getLHfreq(period):
    '''
    Computes the lower hybrid frequency using PlasmaPy from B-field data in a given period
    Inputs:
        period- relevant period
    Outputs:
        LHmean- average lower hybrid frequency calculated across average |B| from all MMS spacecraft
        LHstd- one standard deviation of the array of calculated lower hybrid frequencies
    '''
    #gather |B| field data from each MMS spacecraft
    MMS1x, MMS1y = getData(files[0], period, 'all')
    MMS2x, MMS2y = getData(files[1], period, 'all')
    MMS3x, MMS3y = getData(files[2], period, 'all')
    MMS4x, MMS4y = getData(files[3], period, 'all')
    
    N = len(MMS1x)
    
    #average |B| measurements of all MMS spacecraft
    B = []
    for i in range(0, N):
        avgB = (MMS1y[i] + MMS2y[i] + MMS3y[i] + MMS4y[i])/4
        B.append(avgB)
    
    LHfrequencies = []
    sumLHfreq = 0

    for i in range(0, N):
        fLH = lower_hybrid_frequency(B[i]*u.nT, n_i=0.17*u.cm**-3, ion="p+")
        LHfrequencies.append(fLH.value/(2*math.pi))
        sumLHfreq += (fLH.value)/(2*math.pi)
    
    LHmean = sumLHfreq/N #average LH frequency
    LHstd = np.std(LHfrequencies) #1 standard deviation
    
    return LHmean, LHstd

def FGManalysis(period, axis, ppObj, title, FFTtitle):
    '''
    Plots and generates analysis of MMS spacecraft data
    Inputs:
        period- relevant period
        axis- String value, "x","y","z", or "all" (a vector summation of all components)
        ppObj- PDF file to write plot results to
        title- title of raw signal plot
        FFTtitle- title of FFT plot
    Outputs (not returned):
        pdf- file to which all raw signals and FFT plots generated by Matplotlib are written
        power.txt- file to which all power analysis outputs are written
    '''
    coordinateLabels = {"x":"Bx", "y":"By", "z":"Bz", "all":"|B|"}
    
    MMS1x, MMS1y = getData(files[0], period, axis)
    MMS2x, MMS2y = getData(files[1], period, axis)
    MMS3x, MMS3y = getData(files[2], period, axis)
    MMS4x, MMS4y = getData(files[3], period, axis)

    #Raw Signal
    fig1 = plt.figure(1)

    fig1.autofmt_xdate()
    
    ticks = createTicks(period, MMS1x)
    plt.xticks(ticks[0], ticks[1])  
    
    s1 = plt.plot(MMS1x, MMS1y, '-k')
    s2 = plt.plot(MMS2x , MMS2y, '-r')
    s3 = plt.plot(MMS3x , MMS3y, '-g')
    s4 = plt.plot(MMS4x , MMS4y, '-b')
    
    plt.legend(labels = ('MMS1', 'MMS2', 'MMS3', 'MMS4'), loc = 'lower right')
    plt.title(title)
    plt.xlabel('2017-08-10 12:18 UTC, Epoch (s)')
    plt.ylabel(coordinateLabels.get(axis) + ' Field (nT)')

    #FFT
    MMS1xf, MMS1yf, MMS1N = getFFTdata(MMS1x, MMS1y)
    MMS2xf, MMS2yf, MMS2N = getFFTdata(MMS2x, MMS2y)
    MMS3xf, MMS3yf, MMS3N = getFFTdata(MMS3x, MMS3y)
    MMS4xf, MMS4yf, MMS4N = getFFTdata(MMS4x, MMS4y)

    fig2 = plt.figure(2)
    plt.plot(MMS1xf, 1.0/MMS1N * np.abs(MMS1yf), '-k')
    plt.plot(MMS2xf, 1.0/MMS2N * np.abs(MMS2yf),'-r')
    plt.plot(MMS3xf, 1.0/MMS3N * np.abs(MMS3yf), '-g')
    plt.plot(MMS4xf, 1.0/MMS4N * np.abs(MMS4yf), '-b')
    
    #plot lower hybrid frequency range
    avgLH, stdLH = getLHfreq(period)
    plt.axvline(x=avgLH, color='m', linestyle='--')
    plt.axvspan(avgLH-stdLH, avgLH+stdLH, alpha=0.5, color='y')
    
    plt.yscale('log')
    plt.legend(labels = ('MMS1', 'MMS2', 'MMS3', 'MMS4', 'Avg. LH Freq.', '1 Std. Dev.'), loc = 'upper right')
    plt.title(FFTtitle)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Wave Energy ' + coordinateLabels.get(axis) + '^2 (nT^2)')

    #set FFT window
    plt.xlim([0, 10])
    plt.ylim([10**-2,10**1])

    plt.grid()
    
    #Power Analysis, written to fgmPower.txt
    f = open("./Results/fgmPower.txt", "a")
    indexLH = np.where(abs(MMS1xf-avgLH)<=1)[0][0] #approximate X-index for LH frequency value 
    
    f.write(FFTtitle + " Signal Power (nT^2); < LH Freq.\n")
    lessLH = powerFFT(period, MMS1xf, [MMS1yf, MMS2yf, MMS3yf, MMS4yf], indexLH, "upperLimit")
    f.write(str(lessLH))
    f.write("\n")
    
    f.write(FFTtitle + " Signal Power (nT^2); > LH Freq.\n")
    moreLH = powerFFT(period, MMS1xf, [MMS1yf, MMS2yf, MMS3yf, MMS4yf], indexLH, "lowerLimit") 
    f.write(str(moreLH))
    f.write("\n")
    
    f.write("Ratio of < LH Freq. to > LH Freq.\n")
    f.write(str(lessLH/moreLH))
    f.write("\n")
    f.write("\n")

    f.close()
    
    #save plots to PDF file
    pdf.savefig(fig1)
    pdf.savefig(fig2)
    plt.close('all')

In [7]:
#MAIN

period = [] #start min, start sec, stop min, stop sec
#clear contents from fgmPower.txt
clear = open("./Results/fgmPower.txt", "w")
clear.close()

pdf = PdfPages("./Results/fgmBResults.pdf")

#|B| field PLOTS AND ANALYSIS
FGManalysis([18, 29, 18, 37], "all", pdf, "FGM |B| Field Plot - Full Period", "FGM |B| Field FFT - Full Period")
FGManalysis([18, 29, 18, 31], "all", pdf, "FGM |B| Field Plot - 1/4 Period", "FGM |B| Field FFT - 1/4 Period")
FGManalysis([18, 31, 18, 33], "all", pdf, "FGM |B| Field Plot - 2/4 Period", "FGM |B| Field FFT - 2/4 Period")
FGManalysis([18, 33, 18, 35], "all", pdf, "FGM |B| Field Plot - 3/4 Period", "FGM |B| Field FFT - 3/4 Period")
FGManalysis([18, 35, 18, 37], "all", pdf, "FGM |B| Field Plot - 4/4 Period", "FGM |B| Field FFT - 4/4 Period")

#Bx field PLOTS AND ANALYSIS
FGManalysis([18, 29, 18, 37], "x", pdf, "FGM Bx Field Plot - Full Period", "FGM Bx Field FFT - Full Period")
FGManalysis([18, 29, 18, 31], "x", pdf, "FGM Bx Field Plot - 1/4 Period", "FGM Bx Field FFT - 1/4 Period")
FGManalysis([18, 31, 18, 33], "x", pdf, "FGM Bx Field Plot - 2/4 Period", "FGM Bx Field FFT - 2/4 Period")
FGManalysis([18, 33, 18, 35], "x", pdf, "FGM Bx Field Plot - 3/4 Period", "FGM Bx Field FFT - 3/4 Period")
FGManalysis([18, 35, 18, 37], "x", pdf, "FGM Bx Field Plot - 4/4 Period", "FGM Bx Field FFT - 4/4 Period")

#By field PLOTS AND ANALYSIS
FGManalysis([18, 29, 18, 37], "y", pdf, "FGM By Field Plot - Full Period", "FGM By Field FFT - Full Period")
FGManalysis([18, 29, 18, 31], "y", pdf, "FGM By Field Plot - 1/4 Period", "FGM By Field FFT - 1/4 Period")
FGManalysis([18, 31, 18, 33], "y", pdf, "FGM By Field Plot - 2/4 Period", "FGM By Field FFT - 2/4 Period")
FGManalysis([18, 33, 18, 35], "y", pdf, "FGM By Field Plot - 3/4 Period", "FGM By Field FFT - 3/4 Period")
FGManalysis([18, 35, 18, 37], "y", pdf, "FGM By Field Plot - 4/4 Period", "FGM By Field FFT - 4/4 Period")

#Bz field PLOTS AND ANALYSIS
FGManalysis([18, 29, 18, 37], "z", pdf, "FGM Bz Field Plot - Full Period", "FGM Bz Field FFT - Full Period")
FGManalysis([18, 29, 18, 31], "z", pdf, "FGM Bz Field Plot - 1/4 Period", "FGM Bz Field FFT - 1/4 Period")
FGManalysis([18, 31, 18, 33], "z", pdf, "FGM Bz Field Plot - 2/4 Period", "FGM Bz Field FFT - 2/4 Period")
FGManalysis([18, 33, 18, 35], "z", pdf, "FGM Bz Field Plot - 3/4 Period", "FGM Bz Field FFT - 3/4 Period")
FGManalysis([18, 35, 18, 37], "z", pdf, "FGM Bz Field Plot - 4/4 Period", "FGM Bz Field FFT - 4/4 Period")

pdf.close()

print("Done.")

[-5.3814535, -5.3734517, -5.397957, -5.411113, -5.434789, -5.4989944, -5.4864116, -5.514872, -5.561816, -5.5749674, -5.564173, -5.5813136, -5.641917, -5.6519394, -5.690549, -5.706906, -5.7199364, -5.699821, -5.684765, -5.709724, -5.724371, -5.718186, -5.684817, -5.6869345, -5.7045307, -5.7077966, -5.7087884, -5.7092853, -5.6849504, -5.678108, -5.6765013, -5.692419, -5.702711, -5.7054353, -5.6985955, -5.6517715, -5.6355443, -5.5955553, -5.554451, -5.5277514, -5.522419, -5.521, -5.4990344, -5.492766, -5.4694934, -5.425983, -5.4378834, -5.4450097, -5.492951, -5.5336094, -5.550296, -5.577664, -5.604719, -5.607907, -5.635237, -5.6908245, -5.727197, -5.7351456, -5.717374, -5.7299805, -5.7446346, -5.7491364, -5.7548604, -5.775969, -5.820703, -5.8263154, -5.841889, -5.8468733, -5.8371587, -5.826067, -5.8326306, -5.8358607, -5.8401713, -5.8563395, -5.8818893, -5.8749137, -5.8649936, -5.902556, -5.9191194, -5.923958, -5.9451094, -5.9543023, -5.9413953, -5.927337, -5.957709, -5.9616904, -5.922096